In [1]:
import pandas as pd
import scipy.io as sio
import numpy as np
import math

# Load DVA dataset

In [2]:
vi_dr_indi=pd.read_excel('../DVA/data_raw/database.xlsx', sheet_name='Indications(vi-dr)')
vi_dr_indi=vi_dr_indi[['VIRUS', 'DRUG']]
vi_dr_matrix=vi_dr_indi.groupby(['VIRUS', 'DRUG']).size().unstack().fillna(0) 
vi_dr_matrix=vi_dr_matrix.sort_index(axis=1)
DVA_drug_virus_interaction_df = vi_dr_matrix
drug_names = DVA_drug_virus_interaction_df.columns
virus_names = DVA_drug_virus_interaction_df.index

In [3]:
DVA_drug_drug_sim_df = pd.read_csv('../DVA/data_processed/similarity_drugs.csv')
DVA_drug_drug_sim_df.index = DVA_drug_drug_sim_df['DRUG2'].values
DVA_drug_drug_sim_df = DVA_drug_drug_sim_df.drop(columns=['DRUG2'], axis = 1)
DVA_drug_drug_sim_df

,Abacavir,Acyclovir,Adefovir,Adefovir dipivoxil,Amantadine,Amprenavir,Asunaprevir,Atazanavir,Baloxavir marboxil,Beclabuvir,...,Valaciclovir,Valganciclovir,Valomaciclovir,Vaniprevir,Velpatasvir,Vidarabine,Voxilaprevir,Zalcitabine,Zanamivir,Zidovudine
Abacavir,1.000000,0.501832,0.414474,0.414474,0.156250,0.051724,0.151603,0.053352,0.095238,0.120690,...,0.325967,0.412429,0.287313,0.142061,0.181486,0.525424,0.129235,0.324503,0.195929,0.246459
Acyclovir,0.443662,1.000000,0.551724,0.551724,0.000000,0.018868,0.043478,0.059002,0.127273,0.069277,...,0.694215,0.530249,0.260163,0.027397,0.109741,0.617021,0.062740,0.416309,0.075269,0.302817
Adefovir,0.414474,0.551724,1.000000,1.000000,0.000000,0.052632,0.057143,0.057269,0.103448,0.079882,...,0.423841,0.315789,0.223709,0.026667,0.116505,0.520913,0.059850,0.236749,0.000000,0.171171
Adefovir dipivoxil,0.414474,0.551724,1.000000,1.000000,0.000000,0.052632,0.057143,0.057269,0.103448,0.079882,...,0.423841,0.315789,0.223709,0.026667,0.116505,0.520913,0.059850,0.236749,0.000000,0.171171
Amantadine,0.156250,0.000000,0.000000,0.000000,1.000000,0.061571,0.127695,0.000000,0.017544,0.116667,...,0.000000,0.000000,0.000000,0.092308,0.095541,0.000000,0.091160,0.102662,0.000000,0.122449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vidarabine,0.525424,0.617021,0.520913,0.520913,0.000000,0.050725,0.055556,0.055635,0.064516,0.112760,...,0.465798,0.525974,0.274131,0.074830,0.100703,1.000000,0.109694,0.522634,0.122449,0.404110
Voxilaprevir,0.167315,0.062740,0.059850,0.059850,0.097222,0.185270,0.486486,0.223022,0.180851,0.359003,...,0.057579,0.056257,0.082927,0.551940,0.207646,0.122581,1.000000,0.106613,0.115288,0.112549
Zalcitabine,0.324503,0.416309,0.236749,0.236749,0.102662,0.142241,0.092308,0.059908,0.144465,0.127214,...,0.273885,0.324921,0.121324,0.156250,0.096059,0.522634,0.106613,1.000000,0.130435,0.690141
Zanamivir,0.291209,0.075269,0.000000,0.000000,0.000000,0.151631,0.125541,0.098266,0.000000,0.037736,...,0.080460,0.103604,0.114754,0.128134,0.092150,0.122449,0.128010,0.130435,1.000000,0.143617


In [45]:
sio.savemat('drug_sim_matrix1.mat',{'Sd':DVA_drug_drug_sim_df.values , 'dr_names':DVA_drug_drug_sim_df.index.values})

In [4]:
DVA_virus_virus_sim_df = pd.read_csv('../data/similarity_virus.csv')
DVA_virus_virus_sim_df.index = DVA_virus_virus_sim_df['d2star']
DVA_virus_virus_sim_df = DVA_virus_virus_sim_df.drop(columns=['d2star'], axis = 1)
DVA_virus_virus_sim_df

,Chikungunya virus,Coxsackievirus B5,Ebola virus,Enterovirus D,Enterovirus J,HBV,HCV,HHV-1,HHV-2,HHV-3,...,Nipah virus,RSV,SARS-CoV,SARS-CoV-2,SARS-CoV-2/BHR,SARS-CoV-2/IND2,SARS-CoV-2/USA/AL,VV,Variola virus,Zika virus
d2star,,,,,,,,,,,,,,,,,,,,,
Chikungunya virus,1.000000,0.516603,0.532395,0.519898,0.508609,0.521714,0.539485,0.579950,0.575516,0.548850,...,0.530703,0.523285,0.555768,0.570482,0.571042,0.569493,0.569774,0.582249,0.594740,0.541225
Coxsackievirus B5,0.516603,1.000000,0.513568,0.570859,0.557930,0.516581,0.488933,0.542364,0.527871,0.500370,...,0.541211,0.523202,0.540989,0.553558,0.553633,0.553917,0.554496,0.510400,0.525899,0.501666
Ebola virus,0.532395,0.513568,1.000000,0.503441,0.498397,0.509264,0.535986,0.536519,0.551883,0.529696,...,0.539395,0.520057,0.557383,0.559995,0.560754,0.560548,0.559151,0.542935,0.548989,0.520352
Enterovirus D,0.519898,0.570859,0.503441,1.000000,0.559862,0.522836,0.474324,0.530538,0.520856,0.524424,...,0.513799,0.503035,0.539165,0.538074,0.537502,0.538426,0.537993,0.538473,0.539484,0.488926
Enterovirus J,0.508609,0.557930,0.498397,0.559862,1.000000,0.506899,0.484325,0.502548,0.504797,0.499448,...,0.522267,0.527416,0.548926,0.533067,0.533419,0.532940,0.533549,0.505114,0.510359,0.502136
HBV,0.521714,0.516581,0.509264,0.522836,0.506899,1.000000,0.509182,0.538787,0.544751,0.520635,...,0.527798,0.513848,0.522710,0.525323,0.524589,0.526115,0.524838,0.534670,0.537287,0.526243
HCV,0.539485,0.488933,0.535986,0.474324,0.484325,0.509182,1.000000,0.601197,0.610380,0.565284,...,0.529315,0.525334,0.533091,0.533642,0.533877,0.532928,0.534290,0.555293,0.552597,0.537866
HHV-1,0.579950,0.542364,0.536519,0.530538,0.502548,0.538787,0.601197,1.000000,0.923709,0.664674,...,0.561140,0.533397,0.590993,0.583255,0.582502,0.583773,0.582919,0.620861,0.630542,0.546510
HHV-2,0.575516,0.527871,0.551883,0.520856,0.504797,0.544751,0.610380,0.923709,1.000000,0.654162,...,0.562328,0.533808,0.588871,0.585227,0.585647,0.584823,0.585937,0.621845,0.632663,0.555958


In [5]:
DVA_drug_virus_interaction_df.index = DVA_virus_virus_sim_df.index
virus_names = DVA_drug_virus_interaction_df.index

In [44]:
import scipy.io as sio
sio.savemat('vir_sim_matrix1.mat',{'Sv':DVA_virus_virus_sim_df.values , 'vi_names':DVA_virus_virus_sim_df.index.values})

# WGRMF implementation

In [6]:
import random
import copy
from scipy.sparse.linalg import svds
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from scipy.linalg import sqrtm
from scipy.linalg import fractional_matrix_power

In [256]:
crossval(DVA_drug_virus_interaction_df.values.T, DVA_drug_drug_sim_df.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_p',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10
[9261]
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01


C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in 

****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01
****
k%g		%g	%g	%g		 50 0.25 0.1 0.01


      AUC: %g
 0.8520543478341673
     AUPR: %g
 0.4162240400072469
      AUC: %g
 0.04274790970620938
     AUPR: %g
 0.1062037937003936


C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [272]:
crossval(DVA_drug_virus_interaction_df.values.T, DVA_drug_drug_sim_df.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_d',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10
[2461]
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1


C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in 

****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1


      AUC: %g
 0.7565368387878407
     AUPR: %g
 0.41442910082520595
      AUC: %g
 0.0917955700667107
     AUPR: %g
 0.12186465262842239


C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [271]:
crossval(DVA_drug_virus_interaction_df.values.T, DVA_drug_drug_sim_df.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_v',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10
[8581]
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1


C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in 

****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1
****
k%g		%g	%g	%g		 50 2 0.1 0.1


      AUC: %g
 0.846206608151215
     AUPR: %g
 0.4784031494748323
      AUC: %g
 0.12065475913728002
     AUPR: %g
 0.2047841908685334


C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_17380\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


# Crossval

In [20]:

def crossval(Y,Sd,St,classifier,cv_setting,m,n,use_WKNKN,K,eta,use_W_matrix):
# 
# INPUT:
# Y:           matrix to be modified
# Sd:          pairwise drug similarities matrix
# St:          pairwise target similarities matrix
# classifier:  algorithm to be used for DTI prediction
# cv_setting:  cross validation setting ('cv_d', 'cv_t' or 'cv_p')
# m:           number of repetitions of n-fold cross validation
# n:           number of folds in each n-fold cross validation

#     parameters
    print('m = \n',m)     #repetitions
    print('n = \n',n)     #folds

#     seeds
    seeds = [9261]
#     seeds = []
#     for i in range(m):
#         r = random.randint(1,10000)
#         seeds.append(r)
        
#     print(seeds)
#     print('==========================')



#     cross validation (m repetitions of n-fold experiments)
#     AUCs  = []
#     AUPRs = []
    for i in range(m):
        seed = seeds[i]
#         AUCi, AUPRi = nfold(Y,Sd,St,classifier,n,seed,cv_setting,use_WKNKN,K,eta,use_W_matrix)
        nfold(Y,Sd,St,classifier,n,seed,cv_setting,use_WKNKN,K,eta,use_W_matrix)
#         AUCs.append(AUCi)
#         AUPRs.append(AUPRi)
    


# #     display evaluation results
#     print('\n FINAL AVERAGED RESULTS\n\n')
#     print('     AUC (std): %g\t(%g)\n',   mean(np.array(AUCs)),  std(np.array(AUCs)))
#     print('    AUPR (std): %g\t(%g)\n',   mean(np.array(AUPRs)), std(np.array(AUPRs)))

# nfold

In [33]:
def nfold(Y,Sd,St,classifier,nr_fold,seed,cv_setting,use_WKNKN,K,eta,use_W_matrix):
# nfold is a helper function of crossValidation.m. Depending on the
# specified CV setting (or scenario) and supplied "seed", it divides the
# interaction matrix into "nr_fold" folds, performs a cross validation
# experiment and then reports the results (AUPR/AUC) back to
# crossValidation.m.
# 
# INPUT:
#  Y:           matrix to be modified
#  Sd:          pairwise row similarities matrix
#  St:          pairwise column similarities matrix
#  classifier:     function/script of algorithm to be used for DTI prediction
#  nr_fold:     number of folds in cross validation experiment
#  seed:        seed used for random sampling
#  cv_setting:  drug prediction case or target prediction case
# 
# OUTPUT:
#  auc_res:     AUC result
#  aupr_res:    AUPR result
# 

    num_drugs,num_targets = Y.shape;
    if (cv_setting =='cv_p'):
        num = Y.size
    elif (cv_setting =='cv_d'):
        num = num_drugs
    elif (cv_setting =='cv_v'):
        num = num_targets
    else:
        print('error')
    
#     rng('default')
#     rng(seed);
#     rand_ind = randperm(len);

    AUCs  = []
    AUPRs = []
    for i in range(nr_fold):
#         leave out random drug-target pairs
        global rand_ind
        if (cv_setting == 'cv_p'):
            rand_ind = []
            for d in range(num_drugs):
                for v in range(num_targets):
                    rand_ind.append((d,v))
            random.seed(seed)
            random.shuffle(rand_ind)

#         leave out random entire drugs
#         leave out random entire targets
        elif (cv_setting == 'cv_d' or cv_setting == 'cv_v'):
            rand_ind=np.random.RandomState(seed=seed).permutation(num).tolist()
#             print(rand_ind)
        
        test_ind=rand_ind[(math.floor(np.dot((i),num) / nr_fold)):(math.floor(np.dot(i+1,num) / nr_fold))]
        

#         predict with test set being left out
        y2=copy.deepcopy(Y)
        if (cv_setting == 'cv_p' or cv_setting == 'cv_d'):
            for i in test_ind:
                y2[i] = 0     # test set = ZERO
        elif (cv_setting == 'cv_v'):
            for i in test_ind:
                y2[:,i] = 0
#         print('****')
        if classifier == 'WGRMF':
            y3 = WGRMF(y2,Sd,St,cv_setting,nr_fold,test_ind,use_WKNKN,K,eta,use_W_matrix) # predict!

#         compute evaluation metrics based on obtained prediction scores
        if (cv_setting == 'cv_p'):
            y_true = [Y[i] for i in test_ind]
            y_test = [y3[i] for i in test_ind]
            auci, aupri = returnEvaluationMetrics(y_true, y_test)
        elif (cv_setting == 'cv_d' or cv_setting == 'cv_v'):
            y_true = []
            y_test = []
            for i in test_ind:
                for m in Y[i]:
                    y_true.append(m)
                for n in y3[i]:
                    y_test.append(n)
            auci, aupri = returnEvaluationMetrics(y_true, y_test)
        AUCs.append(auci)
        AUPRs.append(aupri)


    auc_res = np.mean(np.array(AUCs))
    aupr_res = np.mean(np.array(AUPRs))
    auc_res1 = np.std(np.array(AUCs))
    aupr_res1 = np.std(np.array(AUPRs))
    print('\n')
    print('      AUC: %g\n',   auc_res)
    print('     AUPR: %g\n',   aupr_res)
    print('      AUC: %g\n',   auc_res1)
    print('     AUPR: %g\n',   aupr_res1)

# WGRMF

In [31]:
def WGRMF(Y,Sd,St,cv_setting,nr_fold,test_ind,use_WKNKN,K,eta,use_W_matrix):

# INPUT:
#  Y:           interaction matrix
#  Sd:          pairwise drug similarities matrix
#  St:          pairwise target similarities matrix
#  cv_setting:  cross validation setting ('cv_d', 'cv_t' or 'cv_p')
#  nr_fold:     number of folds in cross validation experiment
#  left_out:    if cv_setting=='cv_d' --> left_out is 'drug' indices that are left out
#               if cv_setting=='cv_t' --> left_out is 'target' indices that are left out
#               if cv_setting=='cv_p' --> left_out is 'drug-target pair' indices that are left out
# 
# OUTPUT:
#  y3:  prediction matrix
#     get best parameters
    k = 50
    lambda_l = 2
    lambda_d = 0.1
    lambda_t = 0.1
    p = 5
    num_iter = 2
#     print('k%g\t\t%g\t%g\t%g\t\t',k,lambda_l,lambda_d,lambda_t)
    
#     preprocessing Y
    if use_WKNKN:
        Y = preprocess_WKNKN(Y,Sd,St,K,eta)

#     preprocessing Sd & St
    Sd = preprocess_PNN(Sd,p)
    St = preprocess_PNN(St,p)

#     Laplacian Matrices
    Dd = np.diag(Sd.sum(axis = 0))
    Dt = np.diag(St.sum(axis = 0))
    Ld = Dd - Sd
    tmp = np.matmul(fractional_matrix_power(Dd, -0.5),Ld)
    Ld = np.matmul(tmp,fractional_matrix_power(Dd, -0.5))
    Lt = Dt - St
    tmp1 = np.matmul(fractional_matrix_power(Dt, -0.5),Lt)
    Lt = np.matmul(tmp1,fractional_matrix_power(Dt, -0.5))

#     initialize A & B
    A,B = initializer(Y,k)
    
    W = np.ones(Y.shape)
    for i in test_ind:
        W[i] = 0
    A,B = WGRMF_predict(Y,A,B,Ld,Lt,lambda_l,lambda_d,lambda_t,num_iter,W)
    
#     compute prediction matrix
    y3 = np.matmul(A,B.T)
    return y3

# preprocess_PNN

In [10]:
def preprocess_PNN(s,p):
# preprocess_PNN sparsifies the similarity matrix S by keeping, for each
# drug/target, the p nearest neighbors and discarding the rest.
#     
# S = preprocess_PNN(S,p)

    NN_mat = np.zeros(s.shape);

#     % for each drug/target...
    for j in range(NN_mat.shape[0]):
        row = copy.deepcopy(s[j,:])                           # get row corresponding to current drug/target
        row[j] = 0                             # ignore self-similarity
        indx = row.argsort()[::-1][:p]         # sort similarities descendingly, keep p NNs
        NN_mat[j,indx] = s[j,indx]             # keep similarities to p NNs
        NN_mat[j,j] = s[j,j]                   # also keep the self-similarity (typically 1)

#     symmetrize the modified similarity matrix
    s = (NN_mat+NN_mat.T)/2
    return s

# initializer

In [11]:
def initializer(Y,k):
# initializer initializes the A and B latent feature matrices for the
# WGRMF algorithms.
# 
# [A,B] = initializer(Y,K)
# 
# INPUT:
#  Y:   interaction matrix to be decomposed into latent feature matrices
#  K:   number of latent features
# 
# OUTPUT:
#  A:   latent feature matrix for drugs
#  B:   latent feature matrix for targets
    if k > min(Y.shape):
        k = min(Y.shape)-1
    u,s,v = svds(Y,k)
#     print(u.shape)
#     print(s.shape)
#     print(v.shape)
    s = np.diag(s)
    A = np.matmul(u,(fractional_matrix_power(s, 0.5)))
    B = np.matmul(v.T,(fractional_matrix_power(s, 0.5)))
#     print(A.shape)
#     print(B.shape)
    
    return A,B

# WGRMF predict

In [12]:
def WGRMF_predict(Y,A,B,Ld,Lt,lambda_l,lambda_d,lambda_t,num_iter,W):
# alg_grmf_predict is a helper function of WGRMF that executes the
# alternating least squares method to obtain the solution.
    K = A.shape[1]
    lambda_d_Ld = lambda_d*Ld
    lambda_t_Lt = lambda_t*Lt
    lambda_l_eye_K = lambda_l*np.identity(K)

    H = W*Y;
    for z in range(num_iter):
        A_old = copy.deepcopy(A)
        HB_minus_alpha_Ld_A_old = np.matmul(H,B) - np.matmul(lambda_d_Ld,A_old)
        for a in range(A.shape[0]):
            tmp = np.matmul(B.T,np.diag(W[a,:]))
#             print(A[a,:].shape)
#             print(HB_minus_alpha_Ld_A_old[a,:].shape)
#             print(np.matmul(tmp,B).shape)
#             print(lambda_l_eye_K.shape)
            A[a,:] = np.linalg.solve((np.matmul(tmp,B) + lambda_l_eye_K).T, HB_minus_alpha_Ld_A_old[a,:].T).T

        B_old = B
        HtA_minus_beta_Lt_B_old = np.matmul(H.T,A) - np.matmul(lambda_t_Lt,B_old)
        for b in range(B.shape[0]):
            tmp = np.matmul(A.T,np.diag(W[:,b]))
            B[b,:] = np.linalg.solve((np.matmul(tmp,A) + lambda_l_eye_K).T, HtA_minus_beta_Lt_B_old[b,:].T).T
            
    return A,B

# WKNKN

In [13]:
def preprocess_WKNKN(Y,Sd,St,K,eta):
# preprocess_WKNKN preprocesses the interaction matrix Y by replacing each
# of the 0's (i.e. presumed non-interactions) with a continuous value
# between 0 and 1. For each 0, the K nearest known drugs are used to infer
# a value, the K nearest known targets are used to infer another value, and
# then the average of the two values is used to replace that 0.

# Y = preprocess_WKNKN(Y,Sd,St,K,eta)

# INPUT:
#  Y:   matrix to be modified
#  Sd:  pairwise row similarities matrix
#  St:  pairwise column similarities matrix
#  K:   number of nearest known neighbors to use
#  eta: decay rate

# OUTPUT:
#  Y:   the modified matrix

#     decay values to be used in weighting similarities later
    eta = np.array([eta**i for i in range(K)])

    y2_new1 = np.zeros(Y.shape)
    y2_new2 = np.zeros(Y.shape)
    
    

    empty_rows = np.where(~Y.any(axis=1))   # get indices of empty rows
    empty_cols = np.where(~Y.any(axis=0))   # get indices of empty columns

#     for each drug i...
    for i in range(Sd.shape[0]):
        drug_sim = copy.deepcopy(Sd[i,:]) # get similarities of drug i to other drugs
        drug_sim[i] = 0    # set self-similiraty to ZERO

        drug_sim[empty_rows] = 0  # to drugs of 
        
        indx = drug_sim.argsort()[::-1][:K]     # sort descendingly, keep only similarities of K nearest neighbors 
#                                                  and their indices

#         computed profile of drug i by using its similarities to its K
#         nearest neighbors weighted by the decay values from eta
        drug_sim = copy.deepcopy(Sd[i,:])
        y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
#         print(i)
#         print(y2_new1[i,:])

#     for each target j...
    for j in range(St.shape[0]):
        target_sim = copy.deepcopy(St[j,:]) # get similarities of target j to other targets
        target_sim[j] = 0    # set self-similiraty to ZERO

        target_sim[empty_cols] = 0    # to targets of empty columns

        indx = target_sim.argsort()[::-1][:K]  # sort descendingly, keep only similarities of K nearest neighbors
#                                                 and their indices

#         computed profile of target j by using its similarities to its K
#         nearest neighbors weighted by the decay values from eta
        target_sim = copy.deepcopy(St[j,:])
        y2_new2[:,j] = np.matmul(Y[:,indx] ,(eta * target_sim[indx]).T )/ target_sim[indx].sum(axis = 0)

#     average computed values of the modified 0's from the drug and target
#     sides while preserving the 1's that were already in Y 
    y3 = np.fmax(Y,((y2_new1 + y2_new2)/2))
    return y3

# returnEvaluationMetrics

In [14]:
def returnEvaluationMetrics(Y,scores):
# returnEvaluationMetrics computes the AUC and AUPR from a set of prediction
# scores and their corresponding labels.
# 
# [AUC,AUPR] = returnEvaluationMetrics(Y,scores)
# 
# INPUT:
#  Y:       labels
#  scores:  prediction scores
# 
# OUTPUT:
#  AUC:     area under ROC curve
#  AUPR:    area under precision-recall curve

    
    AUCi = roc_auc_score(Y, scores)
    AUPRi = average_precision_score(Y, scores)
    return AUCi, AUPRi

In [15]:
fused_sim_df = pd.read_csv('fused_sim_drug.csv')
fused_sim_df

,Abacavir,Acyclovir,Adefovir,Adefovir dipivoxil,Amantadine,Amprenavir,Asunaprevir,Atazanavir,Baloxavir marboxil,Beclabuvir,...,Valaciclovir,Valganciclovir,Valomaciclovir,Vaniprevir,Velpatasvir,Vidarabine,Voxilaprevir,Zalcitabine,Zanamivir,Zidovudine
0,0.804857,0.014063,0.014227,0.012874,0.005175,0.005477,0.002829,0.005153,0.002722,0.002752,...,0.009562,0.012644,0.012105,0.002914,0.002876,0.013635,0.002896,0.011846,0.004346,0.010419
1,0.014063,0.877465,0.014083,0.013013,0.003080,0.003867,0.001724,0.003264,0.001772,0.001677,...,0.012785,0.017923,0.016013,0.001834,0.001761,0.015523,0.001796,0.008669,0.003748,0.007465
2,0.014227,0.014083,0.809157,0.017310,0.003406,0.005881,0.002859,0.005223,0.002735,0.002775,...,0.009659,0.013665,0.013019,0.002929,0.002920,0.011509,0.002918,0.008683,0.004607,0.007900
3,0.012874,0.013013,0.017310,0.809594,0.003475,0.006331,0.003707,0.005948,0.002885,0.003143,...,0.009441,0.013194,0.012362,0.003456,0.003383,0.010465,0.003416,0.007881,0.004562,0.007399
4,0.005175,0.003080,0.003406,0.003475,0.938064,0.004015,0.005334,0.004269,0.003828,0.005386,...,0.002354,0.002820,0.002911,0.006125,0.003796,0.003357,0.005901,0.003968,0.006665,0.003848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.013635,0.015523,0.011509,0.010465,0.003357,0.003531,0.001688,0.003239,0.001780,0.001645,...,0.010235,0.012898,0.011290,0.001812,0.001714,0.868632,0.001765,0.011428,0.005282,0.010762
117,0.002896,0.001796,0.002918,0.003416,0.005901,0.005999,0.018994,0.008594,0.005656,0.016828,...,0.001487,0.001785,0.001789,0.019930,0.011066,0.001765,0.875760,0.002894,0.002943,0.002844
118,0.011846,0.008669,0.008683,0.007881,0.003968,0.005388,0.002816,0.005120,0.002777,0.002747,...,0.005596,0.006840,0.006265,0.002932,0.002857,0.011428,0.002894,0.801694,0.004904,0.016712
119,0.004346,0.003748,0.004607,0.004562,0.006665,0.003629,0.002856,0.003592,0.003562,0.002715,...,0.002705,0.003547,0.004130,0.003200,0.002585,0.005282,0.002943,0.004904,0.922754,0.005071


In [22]:
fused_sim_df1 = pd.read_csv('fused_sim_drug1.csv')
fused_sim_df1

,Abacavir,Acyclovir,Adefovir,Adefovir dipivoxil,Amantadine,Amprenavir,Asunaprevir,Atazanavir,Baloxavir marboxil,Beclabuvir,...,Valaciclovir,Valganciclovir,Valomaciclovir,Vaniprevir,Velpatasvir,Vidarabine,Voxilaprevir,Zalcitabine,Zanamivir,Zidovudine
0,1.000000,0.729566,1.000000,1.000000,0.302989,1.000000,0.323866,1.000000,0.240832,0.275594,...,0.325967,0.586864,0.498892,0.292723,0.335166,0.735413,0.301045,1.000000,0.329941,1.000000
1,0.729566,1.000000,0.787659,0.702240,0.025974,0.169811,0.138079,0.181077,0.215095,0.142480,...,0.694215,0.754732,0.600619,0.088825,0.202638,0.765523,0.170885,0.563909,0.198566,0.417816
2,1.000000,0.787659,1.000000,1.000000,0.024390,1.000000,0.157983,1.000000,0.201850,0.160329,...,0.423841,0.570919,0.526331,0.096547,0.199215,0.781387,0.156955,1.000000,0.127273,1.000000
3,1.000000,0.702240,1.000000,1.000000,0.016129,1.000000,0.320649,1.000000,0.260127,0.227101,...,0.423841,0.555263,0.505114,0.207370,0.275981,0.692724,0.258941,1.000000,0.144737,1.000000
4,0.302989,0.025974,0.024390,0.016129,1.000000,0.133758,0.241249,0.000000,0.098294,0.234445,...,0.000000,0.020000,0.020000,0.215385,0.170565,0.094118,0.203017,0.223924,0.086957,0.226919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.735413,0.765523,0.781387,0.692724,0.094118,0.189030,0.174854,0.143710,0.176550,0.218584,...,0.465798,0.659171,0.478094,0.177627,0.216846,1.000000,0.225104,0.693481,0.308832,0.597065
117,0.301045,0.170885,0.156955,0.258941,0.203017,0.372564,0.716186,0.472201,0.405286,0.597307,...,0.057579,0.252019,0.271509,0.796336,0.472012,0.225104,1.000000,0.254785,0.258111,0.261145
118,1.000000,0.563909,1.000000,1.000000,0.223924,1.000000,0.234293,1.000000,0.308991,0.256885,...,0.273885,0.484485,0.329011,0.267934,0.241896,0.693481,0.254785,1.000000,0.300938,1.000000
119,0.329941,0.198566,0.127273,0.144737,0.086957,0.334191,0.248917,0.260578,0.126437,0.167367,...,0.121718,0.299691,0.342526,0.264230,0.232959,0.308832,0.258111,0.300938,1.000000,0.327128


In [40]:
fused_sim_df2 = pd.read_csv('fused_sim_drug2.csv')
fused_sim_df2

,Abacavir,Acyclovir,Adefovir,Adefovir dipivoxil,Amantadine,Amprenavir,Asunaprevir,Atazanavir,Baloxavir marboxil,Beclabuvir,...,Valaciclovir,Valganciclovir,Valomaciclovir,Vaniprevir,Velpatasvir,Vidarabine,Voxilaprevir,Zalcitabine,Zanamivir,Zidovudine
0,1.000000,0.736385,0.609824,0.551532,0.319238,0.259851,0.323866,0.215288,0.240832,0.275594,...,0.325967,0.593647,0.498892,0.306149,0.344066,0.738955,0.312520,0.634924,0.336275,0.477348
1,0.736385,1.000000,0.789181,0.705263,0.043198,0.183194,0.138079,0.200820,0.215095,0.142480,...,0.694215,0.805927,0.600619,0.101944,0.210734,0.767904,0.181210,0.570939,0.204313,0.434222
2,0.609824,0.789181,1.000000,1.000000,0.030858,0.223781,0.157983,0.193629,0.201850,0.160329,...,0.423841,0.572922,0.526331,0.101424,0.202264,0.782220,0.160891,0.422775,0.129619,0.312283
3,0.551532,0.705263,1.000000,1.000000,0.025366,0.238705,0.320649,0.334093,0.260127,0.227101,...,0.423841,0.558203,0.505114,0.213429,0.279884,0.694380,0.263840,0.356224,0.147993,0.291086
4,0.319238,0.043198,0.030858,0.025366,1.000000,0.146668,0.241249,0.022289,0.098294,0.234445,...,0.000000,0.031283,0.020000,0.225830,0.178352,0.102622,0.212193,0.235491,0.093009,0.257200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.738955,0.767904,0.782220,0.694380,0.102622,0.195971,0.174854,0.154670,0.176550,0.218584,...,0.465798,0.661425,0.478094,0.183913,0.221068,1.000000,0.230227,0.696104,0.311463,0.606128
117,0.312520,0.181210,0.160891,0.263840,0.212193,0.379150,0.716186,0.480485,0.405286,0.597307,...,0.057579,0.258083,0.271509,0.798246,0.902531,0.230227,1.000000,0.262606,0.261575,0.281526
118,0.634924,0.570939,0.422775,0.356224,0.235491,0.335203,0.234293,0.170097,0.308991,0.256885,...,0.273885,0.489896,0.329011,0.276819,0.248384,0.696104,0.262606,1.000000,0.305163,0.811619
119,0.336275,0.204313,0.129619,0.147993,0.093009,0.338215,0.248917,0.267261,0.126437,0.167367,...,0.121718,0.302960,0.342526,0.268202,0.235879,0.311463,0.261575,0.305163,1.000000,0.337815


In [41]:
crossval(DVA_drug_virus_interaction_df.values.T, fused_sim_df2.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_p',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in 



      AUC: %g
 0.7982514067603962
     AUPR: %g
 0.3317657048413457
      AUC: %g
 0.03429563999492205
     AUPR: %g
 0.11538410707404435


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [42]:
crossval(DVA_drug_virus_interaction_df.values.T, fused_sim_df2.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_d',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in 



      AUC: %g
 0.8077350255644319
     AUPR: %g
 0.5092637942567595
      AUC: %g
 0.074955609628317
     AUPR: %g
 0.12254347339005102


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [43]:
crossval(DVA_drug_virus_interaction_df.values.T, fused_sim_df2.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_v',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in 



      AUC: %g
 0.8965168604698224
     AUPR: %g
 0.669141181264681
      AUC: %g
 0.09615742383818054
     AUPR: %g
 0.20934125146923613


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [37]:
crossval(DVA_drug_virus_interaction_df.values.T, DVA_drug_drug_sim_df.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_v',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in 



      AUC: %g
 0.8376854039678552
     AUPR: %g
 0.5502536357719955
      AUC: %g
 0.12557381910095844
     AUPR: %g
 0.24962125161652673


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [38]:
crossval(DVA_drug_virus_interaction_df.values.T, DVA_drug_drug_sim_df.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_d',1,10,True,5, 0.7,True)

C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in 

m = 
 1
n = 
 10


      AUC: %g
 0.756472082645995
     AUPR: %g
 0.40287569466573664
      AUC: %g
 0.08345282617592469
     AUPR: %g
 0.0866660509638335


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)


In [39]:
crossval(DVA_drug_virus_interaction_df.values.T, DVA_drug_drug_sim_df.values, DVA_virus_virus_sim_df.values, 'WGRMF', 'cv_p',1,10,True,5, 0.7,True)

m = 
 1
n = 
 10


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in 



      AUC: %g
 0.7774118562042356
     AUPR: %g
 0.32631629577970545
      AUC: %g
 0.03987692623188993
     AUPR: %g
 0.11847852511147479


C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
C:\Users\k1896702\AppData\Local\Temp\ipykernel_10100\4241090288.py:44: RuntimeWarning: invalid value encountered in true_divide
  y2_new1[i,:] = np.matmul((eta * drug_sim[indx]), Y[indx,:]) / drug_sim[indx].sum(axis = 0)
